# Reproducing Growth Metrics

In [8]:
import requests

DJ_PROTOCOL = "http"
DJ_HOST = "localhost"
DJ_PORT = 8000
DJ_URL = f"{DJ_PROTOCOL}://{DJ_HOST}:{DJ_PORT}"
print(DJ_URL)

http://localhost:8000


In [9]:
response = requests.post(
    f"{DJ_URL}/engines/",
    json={
        "name": "TRINO_DIRECT",
        "version": "",
        "uri": "http://prestostatsbackend.master.dataeng.netflix.net:8080",
    }
)
response.json()

response = requests.post(
    f"{DJ_URL}/engines/",
    json={
        "name": "SPARK_SQL",
        "version": "",
        "uri": "",
    }
)
response.json()


{'detail': 'Engine already exists: `SPARK_SQL` version ``'}

In [10]:
response = requests.post(
    f"{DJ_URL}/catalogs/",
    json={
        "name": "prodhive",
        "engines": [{"name": "TRINO_DIRECT", "version": "", "uri": "http://prestostatsbackend.master.dataeng.netflix.net:8080"}],
    }
)
response.json()

{'detail': 'Catalog already exists: `prodhive`'}

In [11]:
response = requests.post(
    f"{DJ_URL}/nodes/source/",
    json={
        "description": "This is the table that connects allocations for AB tests to all of the core metrics.",
        "mode": "published",
        "name": "ab_nm_alloc_f",
        "display_name": "Non-Member Core Metrics with Allocations",
        "catalog": "prodhive",
        "schema_": "dse",
        "table": "ab_nm_alloc_f",
        "columns": [],
    },
)
response.json()


{'message': 'A node with name `ab_nm_alloc_f` already exists.',
 'errors': [],
 'warnings': []}

In [12]:
response = requests.post(
    f"{DJ_URL}/nodes/source/",
    json={
        "description": "",
        "mode": "published",
        "name": "ab_test_detail_d_v2",
        "display_name": "AB Tests",
        "catalog": "prodhive",
        "schema_": "dse",
        "table": "ab_test_detail_d_v2",
    },
)
response.json()


{'node_revision_id': 2,
 'node_id': 2,
 'type': 'source',
 'name': 'ab_test_detail_d_v2',
 'display_name': 'AB Tests',
 'version': 'v1.0',
 'status': 'valid',
 'mode': 'published',
 'catalog': {'id': 1,
  'uuid': 'f0954bec-c172-42a0-bbe8-0e79b92b5eb6',
  'created_at': '2023-03-16T18:54:35.654457+00:00',
  'updated_at': '2023-03-16T18:54:35.654473+00:00',
  'extra_params': {},
  'name': 'prodhive'},
 'schema_': 'dse',
 'table': 'ab_test_detail_d_v2',
 'description': '',
 'query': None,
 'availability': None,
 'columns': [{'name': 'test_id', 'type': 'LONG', 'attributes': []},
  {'name': 'name', 'type': 'STR', 'attributes': []},
  {'name': 'description', 'type': 'STR', 'attributes': []},
  {'name': 'test_type', 'type': 'STR', 'attributes': []},
  {'name': 'alloc_id_type', 'type': 'STR', 'attributes': []},
  {'name': 'default_cell', 'type': 'INT', 'attributes': []},
  {'name': 'is_enabled', 'type': 'BOOL', 'attributes': []},
  {'name': 'cells', 'type': 'MAP[STR, STR]', 'attributes': []},
 

In [13]:
response = requests.post(
    f"{DJ_URL}/nodes/transform/",
    json={
        "description": "Non-Member Base Table",
        "mode": "published",
        "name": "non_member_base",
        "display_name": "Non-Member Base",
        "columns": [],
        "query": """
select
    test_id, 
    cell_id,
    visitor_device_id,
    account_id,
    alloc_group_id,
    allocation_region_date,
    country_iso_code,
    registration_utc_ts_ms,
    mop_utc_ts_ms,
    signup_utc_ts_ms,
    case
      when alloc_membership_status = 0 then 'cookied never members'
      when alloc_membership_status = 1 then 'cookied former members'
      when membership_status = 1
           and alloc_membership_status = 5 then 'un-cookied former members'
      when membership_status = 0
           and alloc_membership_status = 5
           and registration_utc_ts_ms is null then 'un-cookied never members'
      when membership_status = 2 then 'current members'
      else 'non registered members'
    end AS alloc_mem_status,
    coalesce(
    case 
      when (other_properties['isMDS'] = 'true') then 'TV MDS' 
      when (other_properties['netflixClientPlatform'] = 'browser' and other_properties['uaDeviceCategory'] = 'Desktop') then 'Desktop Web'
      when (other_properties['netflixClientPlatform'] = 'browser' and other_properties['uaDeviceCategory'] != 'Desktop') then 'Mobile Web'
      else other_properties['netflixClientPlatform']
    end,
    'NA') AS platform,
    is_bot,
    is_fraud,
    other_properties,
    retention_35d,
    retention_63d,
    retention_98d
from ab_nm_alloc_f f
where membership_status != 2
    and is_bot != 1
    and suspend_resume_flag != 1
""",
    },
)
response.json()



{'node_revision_id': 3,
 'node_id': 3,
 'type': 'transform',
 'name': 'non_member_base',
 'display_name': 'Non-Member Base',
 'version': 'v1.0',
 'status': 'valid',
 'mode': 'published',
 'catalog': {'id': 1,
  'uuid': 'f0954bec-c172-42a0-bbe8-0e79b92b5eb6',
  'created_at': '2023-03-16T18:54:35.654457+00:00',
  'updated_at': '2023-03-16T18:54:35.654473+00:00',
  'extra_params': {},
  'name': 'prodhive'},
 'schema_': None,
 'table': None,
 'description': 'Non-Member Base Table',
 'query': "\nselect\n    test_id, \n    cell_id,\n    visitor_device_id,\n    account_id,\n    alloc_group_id,\n    allocation_region_date,\n    country_iso_code,\n    registration_utc_ts_ms,\n    mop_utc_ts_ms,\n    signup_utc_ts_ms,\n    case\n      when alloc_membership_status = 0 then 'cookied never members'\n      when alloc_membership_status = 1 then 'cookied former members'\n      when membership_status = 1\n           and alloc_membership_status = 5 then 'un-cookied former members'\n      when membership

In [14]:
response = requests.post(
    f"{DJ_URL}/nodes/source/",
    json={
        "description": "",
        "mode": "published",
        "name": "geo_country_d",
        "display_name": "Country Dim Source",
        "catalog": "prodhive",
        "schema_": "dse",
        "table": "geo_country_d",
    },
)
response.json()


{'node_revision_id': 4,
 'node_id': 4,
 'type': 'source',
 'name': 'geo_country_d',
 'display_name': 'Country Dim Source',
 'version': 'v1.0',
 'status': 'valid',
 'mode': 'published',
 'catalog': {'id': 1,
  'uuid': 'f0954bec-c172-42a0-bbe8-0e79b92b5eb6',
  'created_at': '2023-03-16T18:54:35.654457+00:00',
  'updated_at': '2023-03-16T18:54:35.654473+00:00',
  'extra_params': {},
  'name': 'prodhive'},
 'schema_': 'dse',
 'table': 'geo_country_d',
 'description': '',
 'query': None,
 'availability': None,
 'columns': [{'name': 'country_iso_code', 'type': 'STR', 'attributes': []},
  {'name': 'country_sk', 'type': 'INT', 'attributes': []},
  {'name': 'country_desc', 'type': 'STR', 'attributes': []},
  {'name': 'country_full_desc', 'type': 'STR', 'attributes': []},
  {'name': 'company_code', 'type': 'STR', 'attributes': []},
  {'name': 'subregion_sk', 'type': 'INT', 'attributes': []},
  {'name': 'subregion_pd_sk', 'type': 'INT', 'attributes': []},
  {'name': 'content_subregion_sk', 'type'

In [15]:
response = requests.post(
    f"{DJ_URL}/nodes/dimension/",
    json={
        "description": "Country dim",
        "mode": "published",
        "name": "geo_d",
        "display_name": "Geo Country Dim",
        "query": "select country_iso_code, subregion_desc, country_desc from geo_country_d",
    },
)
response.json()


{'node_revision_id': 5,
 'node_id': 5,
 'type': 'dimension',
 'name': 'geo_d',
 'display_name': 'Geo Country Dim',
 'version': 'v1.0',
 'status': 'valid',
 'mode': 'published',
 'catalog': {'id': 1,
  'uuid': 'f0954bec-c172-42a0-bbe8-0e79b92b5eb6',
  'created_at': '2023-03-16T18:54:35.654457+00:00',
  'updated_at': '2023-03-16T18:54:35.654473+00:00',
  'extra_params': {},
  'name': 'prodhive'},
 'schema_': None,
 'table': None,
 'description': 'Country dim',
 'query': 'select country_iso_code, subregion_desc, country_desc from geo_country_d',
 'availability': None,
 'columns': [{'name': 'country_iso_code', 'type': 'STR', 'attributes': []},
  {'name': 'subregion_desc', 'type': 'STR', 'attributes': []},
  {'name': 'country_desc', 'type': 'STR', 'attributes': []}],
 'updated_at': '2023-03-16T18:56:16.586133+00:00',
 'materialization_configs': [],
 'created_at': '2023-03-16T18:56:16.585680+00:00',
 'tags': []}

In [16]:
response = requests.post(
    f"{DJ_URL}/nodes/source/",
    json={
        "description": "AB test plans",
        "mode": "published",
        "name": "ab_test_plan_d_v2",
        "display_name": "AB Test Plans",
        "catalog": "prodhive",
        "schema_": "dse",
        "table": "ab_test_plan_d_v2",
    },
)
response.json()


{'node_revision_id': 6,
 'node_id': 6,
 'type': 'source',
 'name': 'ab_test_plan_d_v2',
 'display_name': 'AB Test Plans',
 'version': 'v1.0',
 'status': 'valid',
 'mode': 'published',
 'catalog': {'id': 1,
  'uuid': 'f0954bec-c172-42a0-bbe8-0e79b92b5eb6',
  'created_at': '2023-03-16T18:54:35.654457+00:00',
  'updated_at': '2023-03-16T18:54:35.654473+00:00',
  'extra_params': {},
  'name': 'prodhive'},
 'schema_': 'dse',
 'table': 'ab_test_plan_d_v2',
 'description': 'AB test plans',
 'query': None,
 'availability': None,
 'columns': [{'name': 'test_id', 'type': 'LONG', 'attributes': []},
  {'name': 'group_id', 'type': 'STR', 'attributes': []},
  {'name': 'name', 'type': 'STR', 'attributes': []},
  {'name': 'plan_type', 'type': 'STR', 'attributes': []},
  {'name': 'plan_action', 'type': 'STR', 'attributes': []},
  {'name': 'plan_status', 'type': 'STR', 'attributes': []},
  {'name': 'plan_reason', 'type': 'STR', 'attributes': []},
  {'name': 'is_compromised', 'type': 'BOOL', 'attributes'

In [17]:
response = requests.post(
    f"{DJ_URL}/nodes/dimension/",
    json={
        "description": "Allocation Groups",
        "mode": "published",
        "name": "group_d",
        "display_name": "AB Test Allocation Groups",
        "query": "select test_id, group_id alloc_group_id, MAX(name) alloc_group_name from ab_test_plan_d_v2 group by 1, 2",
    },
)
response.json()


{'node_revision_id': 7,
 'node_id': 7,
 'type': 'dimension',
 'name': 'group_d',
 'display_name': 'AB Test Allocation Groups',
 'version': 'v1.0',
 'status': 'valid',
 'mode': 'published',
 'catalog': {'id': 1,
  'uuid': 'f0954bec-c172-42a0-bbe8-0e79b92b5eb6',
  'created_at': '2023-03-16T18:54:35.654457+00:00',
  'updated_at': '2023-03-16T18:54:35.654473+00:00',
  'extra_params': {},
  'name': 'prodhive'},
 'schema_': None,
 'table': None,
 'description': 'Allocation Groups',
 'query': 'select test_id, group_id alloc_group_id, MAX(name) alloc_group_name from ab_test_plan_d_v2 group by 1, 2',
 'availability': None,
 'columns': [{'name': 'test_id', 'type': 'LONG', 'attributes': []},
  {'name': 'alloc_group_id', 'type': 'STR', 'attributes': []},
  {'name': 'alloc_group_name', 'type': 'STR', 'attributes': []}],
 'updated_at': '2023-03-16T18:56:22.461966+00:00',
 'materialization_configs': [],
 'created_at': '2023-03-16T18:56:22.461549+00:00',
 'tags': []}

In [18]:
response = requests.post(
    f"{DJ_URL}/nodes/dimension/",
    json={
        "description": "",
        "mode": "published",
        "name": "test_d",
        "display_name": "AB Tests",
        "query": "select test_id, MAX(name) test_name from ab_test_detail_d_v2 group by 1",
    },
)
response.json()


{'node_revision_id': 8,
 'node_id': 8,
 'type': 'dimension',
 'name': 'test_d',
 'display_name': 'AB Tests',
 'version': 'v1.0',
 'status': 'valid',
 'mode': 'published',
 'catalog': {'id': 1,
  'uuid': 'f0954bec-c172-42a0-bbe8-0e79b92b5eb6',
  'created_at': '2023-03-16T18:54:35.654457+00:00',
  'updated_at': '2023-03-16T18:54:35.654473+00:00',
  'extra_params': {},
  'name': 'prodhive'},
 'schema_': None,
 'table': None,
 'description': '',
 'query': 'select test_id, MAX(name) test_name from ab_test_detail_d_v2 group by 1',
 'availability': None,
 'columns': [{'name': 'test_id', 'type': 'LONG', 'attributes': []},
  {'name': 'test_name', 'type': 'STR', 'attributes': []}],
 'updated_at': '2023-03-16T18:56:22.632090+00:00',
 'materialization_configs': [],
 'created_at': '2023-03-16T18:56:22.631715+00:00',
 'tags': []}

In [19]:
response = requests.post(
    f"{DJ_URL}/nodes/non_member_base/columns/test_id/?dimension=test_d&dimension_column=test_id",
)
response.json()



{'message': 'Dimension node test_d has been successfully linked to column test_id on node non_member_base'}

In [20]:
response = requests.post(
    f"{DJ_URL}/nodes/non_member_base/columns/country_iso_code/?dimension=geo_d&dimension_column=country_iso_code",
)
response.json()


{'message': 'Dimension node geo_d has been successfully linked to column country_iso_code on node non_member_base'}

In [21]:
response = requests.post(
    f"{DJ_URL}/nodes/non_member_base/columns/alloc_group_id/?dimension=group_d&dimension_column=alloc_group_id",
)
response.json()


{'message': 'Dimension node group_d has been successfully linked to column alloc_group_id on node non_member_base'}

In [22]:
response = requests.post(
    f"{DJ_URL}/nodes/dimension/",
    json={
        "description": "AB test cells",
        "mode": "published",
        "name": "cell_d39",
        "display_name": "Test Cells",
        "query": """SELECT cast(cell_id as int) cell_id,
              cell_name test_cell_name 
            FROM
             (
                 select test_id,
                        cells 
                  from ab_test_detail_d_v2
             ) tests
             CROSS JOIN UNNEST(cells) AS t (cell_id, cell_name)""",
    },
)
response.json()


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# TODO: Missing cell_d
response = requests.post(
    f"{DJ_URL}/nodes/non_member_base/columns/cell_id/?dimension=cell_d&dimension_column=cell_id",
)
response.json()


In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/metric/",
    json={
        "description": "allocations count",
        "mode": "published",
        "name": "allocations",
        "display_name": "Allocations Count",
        "query": "select SUM(1) AS allocations_cnt from non_member_base",
    },
)
response.json()


In [ ]:

response = requests.post(
    f"{DJ_URL}/nodes/metric/",
    json={
        "description": "",
        "mode": "published",
        "name": "signup_cnt",
        "display_name": "Total Signups",
        "query": "select SUM(CASE WHEN is_fraud != 1 and signup_utc_ts_ms is not null THEN 1 ELSE 0 END) AS signup_cnt from non_member_base",
    },
)
response.json()


In [ ]:

response = requests.post(
    f"{DJ_URL}/nodes/metric/",
    json={
        "description": "",
        "mode": "published",
        "name": "signup_rate",
        "display_name": "Signup Rate",
        "query": "select SUM(CASE WHEN is_fraud != 1 and signup_utc_ts_ms is not null THEN 1 ELSE 0 END) AS signup_rate_num from non_member_base",
    },
)
response.json()


In [ ]:
response = requests.get(
    f"{DJ_URL}/metrics/allocations/sql/?dimensions=test_d.test_id&dimensions=cell_id&dimensions=test_d.test_name&dimensions=group_d.alloc_group_name&dimensions=geo_d.country_desc&dimensions=geo_d.subregion_desc&dimensions=alloc_mem_status&dimensions=platform&filters=test_id%3D45996"
)
print(response.json()['sql'])


In [ ]:
response = requests.get(
    f"{DJ_URL}/data/allocations/?dimensions=test_d.test_id&dimensions=cell_id&dimensions=test_d.test_name&dimensions=group_d.alloc_group_name&dimensions=geo_d.country_desc&dimensions=geo_d.subregion_desc&dimensions=alloc_mem_status&dimensions=platform&filters=test_id%3D45996"
)
response.json()


In [ ]:
response = requests.get(
    f"{DJ_URL}/data/test_d/",
    params={},
)
response.json()
